In [3]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import os

In [4]:
def plot_signal(s, sr, title="Signal", ax=None):
    if ax is None:
        _, ax = plt.subplots()

    t = np.arange(0, len(s) / sr, 1 / sr)

    ax.plot(t, s)
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Amplitude")
    ax.set_title(title)

    return ax

def plot_mel_spectrogram(s, sr, ax=None, title="Mel-frequency spectrogram"):
    if ax is None:
        _, ax = plt.subplots()

    S = librosa.feature.melspectrogram(
        y=s, sr=sr, n_mels=64, hop_length=64, fmax=8000
    )
    S_dB = librosa.power_to_db(S, ref=np.max)

    img = librosa.display.specshow(
        S_dB, x_axis="time", y_axis="mel", sr=sr, fmax=8000, ax=ax
    )
    ax.set(title=title)
    return img

def reduce_noise(y, sr,  output_path, noise_t=0.5):
    n_fft = 512
    hop_length = 64
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    magnitude, phase = np.abs(D), np.angle(D)

    noise_frames = magnitude[:, :int(noise_t * sr / hop_length)]
    noise_profile = np.mean(noise_frames, axis=1, keepdims=True)
    mask = magnitude / (magnitude + noise_profile + 2)
    S_clean = mask * magnitude * np.exp(1j * phase)

    y_clean = librosa.istft(S_clean, hop_length=hop_length)
    sf.write(output_path, y_clean, sr)

def get_noise(clean_path, noisy_path, output_path):
    clean, sr = sf.read(clean_path)
    noisy, _ = sf.read(noisy_path)

    min_len = min(len(clean), len(noisy))
    clean = clean[:min_len]
    noisy = noisy[:min_len]

    alpha = np.dot(noisy, clean) / np.dot(clean, clean)
    print(f"Scaling factor alpha: {alpha:.4f}")

    clean_scaled = alpha * clean

    noise = noisy - clean_scaled
    sf.write(output_path, noise, sr)

In [5]:
file_name = "p226_008.wav"

signal_path = os.path.join("data", "noisy", file_name)
output_clean_path = os.path.join("data", "clean_test", file_name)
output_noise_path = os.path.join("data", "noise_test", file_name)

In [6]:
files = ["p226_001.wav", "p226_003.wav","p226_005.wav", "p226_007.wav", "p226_009.wav"]
for file_name in files:
    signal_path = os.path.join("data", "noisy", file_name)
    output_clean_path = os.path.join("data", "clean_test", file_name)
    output_noise_path = os.path.join("data", "noise_test", file_name)
    s, sr_s = sf.read(signal_path)
    reduce_noise(s, sr_s, noise_t=0.5, output_path=output_clean_path)
    get_noise(clean_path=output_clean_path, noisy_path=signal_path , output_path=output_noise_path)

Scaling factor alpha: 1.2119
Scaling factor alpha: 1.5150
Scaling factor alpha: 1.3456
Scaling factor alpha: 1.3830
Scaling factor alpha: 1.3426
